In [1]:
import pandas as pd

In [31]:
from stop_words import get_stop_words

In [32]:
stopit = get_stop_words('it')
stopen = get_stop_words('en')

In [7]:
df = pd.read_json("friends_1st_level.json", orient='records',lines=False)

In [8]:
df.count()

user_screen_name    14940
screen_name         14940
location            14940
friends_count       14940
listed_count        14940
followers_count     14940
created_at          14940
lang                    0
statuses_count      14940
description         14940
dtype: int64

In [18]:
from nltk.tokenize import WordPunctTokenizer
from nltk.tokenize import TweetTokenizer

In [19]:
tknz = TweetTokenizer()

In [33]:
words = []
for t in df.description.values:
    toks = tknz.tokenize(t)
    for t in toks:
        tl = t.lower()
        tl = tl.replace("#","")
        if(not tl.startswith("@")):
            if(len(tl) > 2 and tl not in stopit and tl not in stopen):
                words.append(tl)
    

In [34]:
from nltk import FreqDist

In [35]:
fq = FreqDist(words)

In [37]:
dfFreq = pd.DataFrame.from_dict(fq, orient='index')
dfFreq.columns = ['counts']

In [67]:
dfFreq.sort_values(by='counts', ascending=False).head(820).tail(30)


,counts
2007,26
protection,26
champion,26
nonprofit,26
copywriter,26
scrivere,26
girl,26
libertà,26
cerco,26
professionale,26


**Proposta Filtro**

In [69]:
paroleFiltro = ['energia', 'energy', 'ambiente', 'green','innovation', 'sostenibile', 'gas', 'clean', 'wind','efficiency',
                'environment', 'economy', 'ambientale', 'energetica', 'rinnovabile', 'rinnovabili','fotovoltaico','sostenibili',
                'renewable', 'renewables','energie', 'environmental', 'elecricity', 'efficienza', 'solar','ecologia',
                'water', 'risparmio','energetico','energetici', 'climate', 'climatico', 'circulareconomy','legambiente',
                'riciclo','ecosostenibile','solare', 'termico', 'eolico', 'turbine', 'turbina','eolica']

In [70]:
df.head()

,user_screen_name,screen_name,location,friends_count,listed_count,followers_count,created_at,lang,statuses_count,description
0,GruppoHera,andreacreativo,"Roma, Lazio",4800,1280,9481,2010-06-01 10:43:33+00:00,NaN,42179,"Scrivo, leggo e twitto di #StartUp #Industria4..."
1,GruppoHera,reportingMerola,,3309,119,1628,2013-11-23 14:20:48+00:00,NaN,1919,"Journalist, Communication manager @WSENSE_IoUT..."
2,GruppoHera,UN_Water,,2557,1321,74188,2010-11-16 13:57:19+00:00,NaN,20828,United Nations Water: Coordinates the UN’s wor...
3,GruppoHera,la_Biennale,"Venice, Italy",5348,3220,707046,2010-03-13 17:52:46+00:00,NaN,23507,Official Twitter page of La Biennale di Venezi...
4,GruppoHera,VitodeCeglia,,59,5,91,2011-12-16 14:56:06+00:00,NaN,91,


**Filtro Utenti**

In [116]:
energies = []
for i,t in enumerate(df.description.values):
    toks = tknz.tokenize(t)
    e = 0
    for t in toks:
        tl = t.lower()
        if(tl in paroleFiltro ):
            e = 1
    if(e):
        energies.append(e)
    else:
        energies.append(e)

In [117]:
len(df)

14940

In [118]:
len(energies)

14940

In [119]:
df = df.assign(energy = energies)

In [121]:
dfe = df[df.energy == 1]

In [123]:
dfe.to_csv("EnergyFirstLevel.csv", index = False)

In [124]:
len(dfe)

2039

In [84]:
df = df.assign(label = "no")

In [85]:
ndf = df[['description','label']]

In [86]:
ndf = ndf.assign(text = ndf.description)

In [87]:
ndf = ndf[['text','label']]

In [93]:
ndf = ndf.assign(text = ndf.text.str.replace("\n+", " ", regex=True))

In [98]:
def getL(s):
    return len(s)

In [99]:
ndf = ndf.assign(lr = ndf.text.apply(getL))

In [101]:
ndfne = ndf[ndf.lr > 0]

In [105]:
ndfne = ndfne[['text', 'label']]

In [106]:
ndfne.to_csv("OnlyDescriptions.csv", index=False)

In [108]:
ndfne = ndfne.assign(text = ndfne.text.str.replace('"', " ", regex=True))
ndfne = ndfne.assign(text = ndfne.text.str.replace("'", " ", regex=True))

In [109]:
ndfne.text.to_csv("OnlyDescriptions.txt", index=False, )

/opt/conda/lib/python3.7/site-packages/ipykernel_launcher.py:1: FutureWarning: The signature of `Series.to_csv` was aligned to that of `DataFrame.to_csv`, and argument 'header' will change its default value from False to True: please pass an explicit value to suppress this warning.
  """Entry point for launching an IPython kernel.


In [111]:
ndfne.to_json("TD.json", lines=True, orient="records")

In [95]:
ndf.to_excel("OnlyDescriptions.xlsx", index=False)